## 1.0 CTS GENERATE

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


u_train = pd.read_csv(os.path.join("data", "dataBenchmark.csv"),header=0, thousands=',', decimal='.', na_values='NAN', usecols=['uEst']).to_numpy()
u_test = pd.read_csv(os.path.join("data", "dataBenchmark.csv"),header=0, thousands=',', decimal='.', na_values='NAN', usecols=['uVal']).to_numpy()
y_train = pd.read_csv(os.path.join("data", "dataBenchmark.csv"),header=0, thousands=',', decimal='.', na_values='NAN', usecols=['yEst']).to_numpy()
y_test = pd.read_csv(os.path.join("data", "dataBenchmark.csv"),header=0, thousands=',', decimal='.', na_values='NAN', usecols=['yVal']).to_numpy()

print(u_train[0])
# Compute rescaling factors using the training data
u_mean = np.mean(u_train)
u_std = np.std(u_train)
y_mean = np.mean(y_train)
y_std = np.std(y_train)

# Rescale variables
#u_train = (u_train - u_mean)/u_std
#u_test = (u_test - u_mean) / u_std

#y_train = (y_train - y_mean)/u_std
#y_test = (y_test - y_mean) / u_std

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(u_train)  # u[V]
ax[1].plot(y_train)  # h1 [m]

if not os.path.exists(os.path.join("data", "CTS")):
    os.makedirs(os.path.join("data", "CTS"))

np.save(os.path.join("data", "CTS", "u_train.npy"), u_train)
np.save(os.path.join("data", "CTS", "u_test.npy"), u_test)
np.save(os.path.join("data", "CTS", "y_train.npy"), y_train)
np.save(os.path.join("data", "CTS", "y_test.npy"), y_test)

# Generate transfer and evaluation datasets


# Rescale variables
#u_transf = (u_transf - u_mean) / u_std
#u_eval = (u_eval - u_mean) / u_std

#y_transf = (y_transf - y_mean) / y_std
#y_eval = (y_eval - y_mean) / y_std

#fig, ax = plt.subplots(2, 1, sharex=True)
#ax[0].plot(u_transf)  # u[V]
#ax[1].plot(y_transf)  # h1 [m]

#np.save(os.path.join("data", "CTS", "u_transf.npy"), u_transf)
#np.save(os.path.join("data", "CTS", "u_eval.npy"), u_eval)
#np.save(os.path.join("data", "CTS", "y_transf.npy"),y_transf)
#np.save(os.path.join("data", "CTS", "y_eval.npy"), y_eval)

## 1.1 CTS TRAIN

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from open_lstm import OpenLSTM
from torchid import metrics
from torch.utils.data import DataLoader, TensorDataset
import os
import numpy as np
import matplotlib.pyplot as plt

# Set seed for reproducibility
n_context = 100  # Used to estimate the initial state in the RNN. State estimation is performed by opening the output prediction loop for the first n_context steps
np.random.seed(0)
torch.manual_seed(0)

num_iter = 10000  # gradient-based optimization steps
#num_iter = 100  # gradient-based optimization steps
lr = 1e-3  # learning rate
test_freq = 10  # print a message every test_freq iterations

# Load data (assuming data is two-dimensional)
u_train = torch.tensor(np.load(os.path.join("data", "CTS", "u_train.npy")).astype(np.float32))  # shape: [sequence_length, num_features]
y_train = torch.tensor(np.load(os.path.join("data", "CTS", "y_train.npy")).astype(np.float32))  # shape: [sequence_length, num_features]

# Add batch dimension
u_train = u_train.unsqueeze(0)  # Shape: [1, sequence_length, num_features]
y_train = y_train.unsqueeze(0)  # Shape: [1, sequence_length, num_features]

# Reshape into sequences of length = sequence_length
sequence_length = 1024  # Adjust as needed
num_features = u_train.shape[-1]  # Automatically determine number of features
u_train = u_train.view(-1, sequence_length, num_features)  # Shape: [batch_size, sequence_length, num_features]
y_train = y_train.view(-1, sequence_length, num_features)  # Shape: [batch_size, sequence_length, num_features]

# Create a TensorDataset
dataset = TensorDataset(u_train, y_train)

# Create DataLoader with batch_size = 1024/sequence_length (1024 is the total size of the dataset)
batch_size = round(1024/sequence_length)  # Adjust batch size if needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Iterate over batches
for u_batch, y_batch in dataloader:
    # Each batch will have shape [batch_size, sequence_length, num_features]
    u_train = u_batch.view(-1, num_features)  # Flatten to [batch_size * sequence_length, num_features]
    y_train = y_batch.view(-1, num_features)  # Flatten to [batch_size * sequence_length, num_features]
    
    print(u_train.shape)  # Expected: [batch_size * sequence_length, num_features]
    print(y_train.shape)  # Expected: [batch_size * sequence_length, num_features]

n_inputs = u_train.shape[-1]  # num_features

# Concatenate the input and output for the model
u_train = torch.cat((u_train[:, :], y_train[:, :]), -1)  # Concatenate along the feature dimension 
y_train = y_train[:, :]  # Adjust target by shifting 

#print(len(u_train.shape))
# Initialize the model
model = OpenLSTM(n_context, n_inputs,sequence_length)# 

# Define loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

LOSS = []
y_sim = []
start_time = time.time()

# Training loop
for itr in range(num_iter):
    optimizer.zero_grad()

    y_sim = model(u_train)  # Model expects input of shape [batch_size * sequence_length, num_features]
    loss = loss_fn(y_sim, y_train) #
    loss.backward()

    LOSS.append(loss.item())
    if itr % test_freq == 0:
        with torch.no_grad():
            print(f'Iter {itr} | Train Loss {loss:.4f}')

    optimizer.step()

train_time = time.time() - start_time
print(f"\nTrain time: {train_time:.2f}")

# Save model
if not os.path.exists("models"):
    os.makedirs("models")

model_name = "lstm-batched"
model_filename = f"{model_name}.pt"
torch.save(model.state_dict(), os.path.join("models", model_filename))

R_sq_lin = metrics.r_squared(y_train[n_context:, :].detach().numpy(), y_sim[n_context:, :].detach().numpy()) #
print(f"R-squared Train model: {np.mean(R_sq_lin, axis=0)}")

import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 1, sharex=True)
plt.suptitle("Train")

# Concatenate all sequences along the time axis
y_train_np = y_train.detach().numpy()
y_sim_np = y_sim.detach().numpy()
u_train_np = u_train.detach().numpy()

# Plot the concatenated sequences
ax[0].plot(y_train_np[:, 0], label='Ground truth')
ax[0].plot(y_sim_np[:, 0], label='Estimated fit')
ax[0].axvline(n_context-1, color='k', linestyle='--', alpha=0.2)
ax[0].set_ylabel('Y')
ax[0].set_xlabel('X')
ax[0].legend()

ax[1].plot(u_train_np[:, 0], label=' u (Input Train)')
ax[1].axvline(n_context-1, color='k', linestyle='--', alpha=0.2)
ax[1].set_ylabel('Y')
ax[1].set_xlabel('X')
ax[1].legend()


plt.show()

# Test the model
u_test = torch.tensor(np.load(os.path.join("data", "CTS", "u_test.npy")).astype(np.float32))
y_test = torch.tensor(np.load(os.path.join("data", "CTS", "y_test.npy")).astype(np.float32))

u_test = u_test.unsqueeze(0)  # Shape: [1, sequence_length, num_features]
y_test = y_test.unsqueeze(0)  # Shape: [1, sequence_length, num_features]

u_test = u_test.view(-1, sequence_length, num_features)  
y_test = y_test.view(-1, sequence_length, num_features)  

u_test = u_test.view(batch_size*sequence_length, -1)  # Flatten to [batch_size * sequence_length, num_features]
y_test = y_test.view(batch_size*sequence_length, -1)  # Flatten to [batch_size * sequence_length, num_features]

u_test = torch.cat((u_test[:, :], y_test[:, :]), -1)  # Concatenate along the feature dimension 
y_test = y_test[:, :] # 

with torch.no_grad():
    y_sim = model(u_test)

aux_batch_size_sequence_length = y_test.shape[0]
R_sq_lin = metrics.r_squared(y_test[n_context:, :].detach().numpy(), y_sim[n_context:, :].detach().numpy()) #
print(f"R-squared Test model: {np.mean(R_sq_lin, axis=0)}")

# Plot test results
fig, ax = plt.subplots(2, 1, sharex=True)
plt.suptitle("Test")

# Concatenate all sequences along the time axis
y_test_np = y_test.detach().numpy()
y_sim_np = y_sim.detach().numpy()
u_test_np = u_test.detach().numpy()

# Plot the concatenated sequences
ax[0].plot(y_test_np[:, 0], label='Ground truth')
ax[0].plot(y_sim_np[:, 0], label='Estimated fit')
ax[0].axvline(n_context - 1, color='k', linestyle='--', alpha=0.2)
ax[0].set_ylabel('Y')
ax[0].set_xlabel('X')
ax[0].legend()

ax[1].plot(u_test_np[:, 0], label='u (Input Test)')
ax[1].axvline(n_context - 1, color='k', linestyle='--', alpha=0.2)
ax[1].set_ylabel('Y')
ax[1].set_xlabel('X')
ax[1].legend()

plt.show()



# Construct RealTime L4CasADi Model from PyTorch Model

In [4]:
import l4casadi as l4c
import os
import casadi as cs
import torch
from open_lstm_l4casadi import OpenLSTML4casadi
import numpy as np

# Initialize the model architecture as before
n_context = 1  # Used for initial state estimation
n_inputs = 1
sequence_length = 1024  # Adjust as needed
batch_size = 1
# Initialize the model
model = OpenLSTML4casadi(n_context, n_inputs, batch_size ,sequence_length, is_estimator=False)

# Load the saved model weights
model_name = "lstm-batched"
model_filename = f"{model_name}.pt"

assert os.path.exists(os.path.join("models", model_filename)), "Model file not found!"
model.load_state_dict(torch.load(os.path.join("models", model_filename), map_location="cpu"))

model.estimate_state(torch.from_numpy(np.asarray(1.9914).reshape(1,1,1)).to(torch.float32),torch.from_numpy(np.asarray(5.122112274169922).reshape(1,1,1)).to(torch.float32),1)

# Convert the PyTorch model to an L4CasADi model with batching
model_l4c = l4c.realtime.RealTimeL4CasADi(model, device='cpu', approximation_order = 1, name="lstm_l4casadi_batched_realtime_rev1")

# Define the CasADi symbolic variable correctly
# The shape should be [sequence_length, batch_size, n_inputs] for batched LSTM

# Create a 2D symbolic matrix with the correct dimensions for LSTM input
u_sym = cs.MX.sym('u', batch_size*sequence_length, n_inputs)
print(u_sym.shape)
y_sym = model_l4c(u_sym)
f = cs.Function('f_approx', [u_sym, model_l4c.get_sym_params()], [y_sym])

casadi_param = model_l4c.get_params(np.ones((n_inputs,batch_size*sequence_length)))
print(f)

(1024, 1)


RuntimeError: output with shape [1, 512] doesn't match the broadcast shape [1, 1, 512]